# Some K-Means Clustering Problems #
## and ##
# A Neural Network Problem #

## Problem 1 ##
This problem has to do with extending the k-Means clustering class (from [here](https://towardsdatascience.com/k-means-clustering-algorithm-applications-evaluation-methods-and-drawbacks-aa03e644b48a)) which was discussed in class and is reproduced below.  The key method in the class is (as usual) ".fit()", which establishes centroids for each cluster and labels for each point in the data set--i.e., which cluster each point belongs to.

Your task in this problem will be to modify the class to produce additional output. 

**Note:** The k-means clustering algorithm is *not* guaranteed to converge, so you may not always get the outcome you expect.

In [78]:
import numpy as np
from numpy.linalg import norm


class Kmeans:
    '''Implementing Kmeans algorithm.'''

    def __init__(self, n_clusters, max_iter=100, random_state=123):
        self.n_clusters = n_clusters
        self.max_iter = max_iter
        self.random_state = random_state

    def initializ_centroids(self, X):
        np.random.RandomState(self.random_state)
        random_idx = np.random.permutation(X.shape[0])
        centroids = X[random_idx[:self.n_clusters]]
        return centroids

    def compute_centroids(self, X, labels):
        centroids = np.zeros((self.n_clusters, X.shape[1]))
        for k in range(self.n_clusters):
            centroids[k, :] = np.mean(X[labels == k, :], axis=0)
        return centroids

    def compute_distance(self, X, centroids):
        distance = np.zeros((X.shape[0], self.n_clusters))
        for k in range(self.n_clusters):
            row_norm = norm(X - centroids[k, :], axis=1)
            distance[:, k] = np.square(row_norm)
        return distance

    def find_closest_cluster(self, distance):
        return np.argmin(distance, axis=1)

    def compute_sse(self, X, labels, centroids):
        distance = np.zeros(X.shape[0])
        for k in range(self.n_clusters):
            distance[labels == k] = norm(X[labels == k] - centroids[k], axis=1)
        return np.sum(np.square(distance))
    
    def fit(self, X):
        self.centroids = self.initializ_centroids(X)
        for i in range(self.max_iter):
            old_centroids = self.centroids
            distance = self.compute_distance(X, old_centroids)
            self.labels = self.find_closest_cluster(distance)
            self.centroids = self.compute_centroids(X, self.labels)
            if np.all(old_centroids == self.centroids):
                break
        self.error = self.compute_sse(X, self.labels, self.centroids)
    
    def predict(self, X):
        distance = self.compute_distance(X, self.centroids)
        return self.find_closest_cluster(distance)

1. Consider the dataset 02-02prob1.csv with two feature variables and available on the github page. Produce a scatter plot of the data and visually estimate the number of clusters in the datset.

2. Consider the class Kmeans defined above, and in particular its ".fit()" method.  Some questions to answer:
+ What are the parameters for the class that can be changed on initialization?
+ What are the stopping criteria (note plural) for the method?
+ What are the three attributes that the method produces/updates?  Give a description of each.

3. The class is very general, and can work with clusters in arbitrary dimensions.  But, as written, the class consists primarily of internal methods and, once fitted, can only be used to predict which cluster a new value of the feature variables belongs to (using ".predict()").  Write a method  .print_centroids()for the class which returns the centroids.

4. Write another method .display_cluster() which (1) produces a scatterplot of the clusters with (2) the color of each point indicating the cluster it belongs to and (3) the centroids of each cluster also displayed in **black**.

5. The k-means clustering algorithm progresses by randomly choosing starting centroids from among the points and then alternatively labelling data points according to the nearest centroid and then updating the centroids to reflect these new clusters--repeating these two steps until a stopping criterion is met.  Write a method .display_fit() for the class which finds the clusters as in the ".fit()" method, but also produces a scatterplot like the one in the previous part *for each iteration* of the algorithm--the goal being to visually display how the cluster classification progresses as the algorithm progresses through each step. 

## Problem  2 ##

The k-means clustering algorithm assumes that the number $k$ of clusters is already known.  But for high-dimensional data, it may not be easy to determine $k$.  One technique for estimating the number of clusters (see p. 313 of the textbook) is the "elbow method".   

The idea is that the "SSE error", which gives the total sum of squares of distances from each point in the data to the centroid of its cluster (see the Class definition above), can be used to estimate the number of centers as follows: For each choice in a range of possible values of $k$, plot the SSE produced by the k-means algorithm for that $k$.  The actual number of clusters should be the $k$ which occurs at the 'elbow' of that plot.  Why?  Geometrically, if a centroid is located at the center of its actual cluster, it should be almost as small as possible. When points from other clusters are included in the cluster (e.g., when $k$ is too small), the SSE will be too big, because the centroid is associated with points that are too far away.  When $k$ is too big, the SSE will be smaller but not by much, since you are simply dividing clusters into smaller pieces and SSEs between the unneeded clusters won't be significantly different.  

See [this link](https://www.scikit-yb.org/en/latest/api/cluster/elbow.html) for a discussion of how this can be implemented with SKLearn.

**Your assignment:**

1. Consider the data set 02-02prob2.csv.  This is a blob of an unspecified number of clusters in 7-dimensional space.  Write a function elbow(n), which produces an elbow plot for the points (k,SSE(k)) for each k between 1 and n, and use it to estimate the number of clusters is my blob.  **The function should make use of the Kmeans class above**, which may need to be modified to produce the needed output.

**Note:** The number of clusters in the blob may not be well-defined.  I've used the SKLearn blob function to create clusters which, when plotted, clearly result in fewer clusters than I've specified.  Hence the nature of randomness!

## Problem 3 (Neural Network) ##

The goal on this problem is to use the Neural Network described in class and in Chapter 11 of the text to classify each of the 10 types of  objects of clothing in the Fashion-MNIST dataset.   Fashion-MNIST--and how to download it--was described in the 01-25-24assn.ipynb notebook [on the Github](https://github.com/aleahy-work/CS-STAT323-W24/tree/main).  The authors neural network was described in the 02-08-24class.ipynb notebook, in the neuralnet-implement.mp4 video posted [on the Classroom page](https://classroom.google.com/c/NjQ2NzA1MzI5OTgz), and in Chapter 11 of the textbook (pp. 347-360). (See 'Some Scikit-Learn Problems' in the Classroom page and 01-25-24assn.ipynb on the Github for the actual assignment.) The expectation is that you should be about to use notes from class or the textbook author's [Jupyter notebook for Chapter 11](https://github.com/rasbt/machine-learning-book/tree/main/ch11) with some minor modifications to load the Fashion-MNIST data, train the author's neural network, and test your model to see its accuracy.  Here are the steps in the problem:

1. Load the Fashion-MNIST **train** dataset.  Normalize the data for use in a neural network and set aside **6000** data points in the train dataset to create a validation set to test training during each epoch.

2. Import or copy-and-paste the author's NeuralNetMLP class and associated functions and create a model which instantiates his NeuralNetMLP class.  (You won't need to change the 'num_features' and 'num_classes' attributes, but you might want to experiment with the number 'num_hidden' of weights in the hidden layer.) Then **train** the model for 50 epochs using (mini-)batches of size 100, as the author did in his example. Depending on your computer, this may take a while, so have fun watching your training and validation accuracy increase with each epoch! **Question:** What is your training and validation accuracy on the last epoch?  (FWIW, mine came in around 88%--slightly lower than the MNIST example the author did.)

3. Evaluate your model's performance by producing plots of mean-squared-error and model accuracy, **and** load the MNIST-Fashion test dataset to see the final test accuracy.  (**Warning:** this is called 't10k', not 'test', in the dataset.  Also, remember to normalize your test data or your test accuracy will be significantly lower than your train and validation accuracy!)